In [3]:
import pandas as pd
import os

In [4]:
all_sheets = pd.read_excel('data/data_raw/regional_gross_disposable_household_income_local_authorities_2023.xlsx', sheet_name=['Table 3', 'Table 4', 'Table 6'], header=1)


In [5]:
df1, df2, df3 = [
    all_sheets[k].rename(columns={
        'LAD code': 'ons_code',
        'Region name': 'ons_geography',
        'Region': 'region'
    }) for k in all_sheets
]

In [6]:
def apply_geo_tags(df):
    df = df.assign(geo_level = 'LAD')
    def get_country(code):
        if code.startswith('E'): return 'England'
        if code.startswith('S'): return 'Scotland'
        if code.startswith('W'): return 'Wales'
        if code.startswith('N'): return 'Northern Ireland'
        return 'Unknown'

    df = df.assign(country = df['ons_code'].apply(get_country))

    df = df.assign(national=df['country'].apply(
        lambda x: 'UK' if x == 'Northern Ireland' else 'GB'
    ))
    def get_county(row):
        code = row['ons_code']
        name = row['ons_geography']
        # Map London boroughs to Greater London
        if code.startswith('E09'): return 'Greater London'
        # Map unitary authorities and regions (Scotland, Wales, NI) to themselves
        if code.startswith(('E06', 'S12', 'W06', 'N09')): return name
        # Use 'region' as a placeholder for England two-tier areas (E07/E08)
        return row['region']

    df = df.assign(county = df.apply(get_county, axis=1))
    return df

df1, df2, df3 = [apply_geo_tags(df) for df in [df1, df2, df3]]


In [7]:
id_cols = ['ons_code', 'ons_geography', 'geo_level', 'national', 'country', 'region', 'county']

# Table 3 (Absolute values): GDHI per head of population, current basic prices (£)
df1_long = df1.melt(id_vars=id_cols, var_name='year', value_name='gdhi_per_head')

# Table 4 (Indices) - Note: GDHI per head indices (UK = 100)
df2_long = df2.melt(id_vars=id_cols, var_name='year', value_name='gdhi_index')

# Table 6 (Growth rates): Annual growth in GDHI per head of population (%)
df3_long = df3.melt(id_vars=id_cols, var_name='year', value_name='gdhi_growth')

# Chain merging (based on df1_long 1997-2023)
# Use left join to ensure 1997 rows are not dropped
master_economic_df = df1_long.merge(
    df2_long, on=['ons_code', 'year'] + id_cols[1:], how='left'
).merge(
    df3_long, on=['ons_code', 'year'] + id_cols[1:], how='left'
)

master_economic_df['year'] = master_economic_df['year'].astype(int)

In [8]:
output_dir = 'data/data_cleaned'
file_name = 'GDHI_LAD_Panel_1997-2023.csv'
output_path = os.path.join(output_dir,file_name)
master_economic_df.to_csv(output_path, index=False)